In [61]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from keras import backend as K
import seaborn as sns
import multiprocessing as mp
import tensorflow as tf
import datetime
import os


def normalize(state, env):
    return (state-env.state_space_low)/(env.state_space_high-env.state_space_low)


def de_normalize(state, env):
    return state*(env.state_space_high-env.state_space_low)+env.state_space_low

# Define the Q-learning agent class
class QLearningAgent:
    def __init__(self, alpha, gamma, epsilon, epsilon_decay, alpha_decay, get_state, action_space):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.alpha_decay = alpha_decay
        self.get_state = get_state
        self.action_space = action_space
        self.num_actions = len(action_space)

        
        self.v_get_Q_value = np.vectorize(self.get_Q_value)
        self.v_get_next_action = np.vectorize(self.get_next_action)

    def gen_Q_table(self, action_space, states):
        
        self.Q = pd.DataFrame(np.random.random((len(action_space), len(states)))*(-0.1),
                            columns=states,
                            index=action_space)

    def get_Q_value(self, state, action):
        if state is None:
            return 0.0
        
        if state in self.Q.columns and action in self.Q.index:
            return self.Q[state][action]
        
        return 0.0

    def get_next_action(self, max_Q_value, Q_values):
        if Q_values == max_Q_value:
            return 1
        else:
            return 0

        
    def choose_action(self, state, greedy=False):

        if greedy and random.random() < self.epsilon:
            action = random.choice(self.action_space)
        else:
           
            Q_values = self.v_get_Q_value(state, self.action_space)

            max_Q_value = max(Q_values)
          
            #action = random.choice([action for i, action in enumerate(self.action_space) if Q_values[i] == max_Q_value])
            action = self.action_space[random.choice(np.where(self.v_get_next_action(max_Q_value, Q_values))[0])]
            
        return action

    def update_Q_value(self, state, action, reward):

        #Q_next = [self.get_Q_value(next_state, next_action) for next_action in self.action_space]
        
        Q_next = self.v_get_Q_value(state, self.action_space)

        max_Q_next = max(Q_next) if len(Q_next) else 0.0
        current_Q = self.get_Q_value(state, action)
        self.Q[state][action] = current_Q + self.alpha * (reward + self.gamma * max_Q_next - current_Q)

    def test(self, num_episodes, num_steps, sp_steps):
        del_next_state = 0
        action = 0
        state = 0
        state_space_high = 40
        state_space_low = -10
        LOG_DIR = 'logs'
        colors = mcolors.TABLEAU_COLORS


        env = Environment()
        
        sp_list = [np.random.random()*state_space_high+state_space_low for i in range(num_steps//sp_steps)]
        states = np.zeros((num_steps, num_episodes))
        actions = np.zeros((num_steps, num_episodes))

        for episode in range(num_episodes):

            states = np.zeros((num_steps, num_episodes))
            actions = np.zeros((num_steps, num_episodes))
            del_next_state = env.reset()
            state = 0
            action = 0
            
            for step in range(num_steps):
                SP = sp_list[step//(sp_steps+1)]

                del_action = agent.choose_action(del_next_state, greedy=True)
                action += del_action
                action = np.round(np.clip(action,env.action_space_low, env.action_space_high), 1)

                del_next_state, reward = env.step(action, SP)
                state += del_next_state
                state = np.round(np.clip(state, state_space_low, state_space_high), 1)
                
                states[step][episode] = state
                actions[step][episode] = action

            if del_next_state is None:
                print('del_next_state None')
                if state-(env.state_space_low+env.state_space_high)/2 < 0:
                    del_next_state = env.state_space_low
                else:
                    del_next_state = env.state_space_high

        
        fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(12, 7))
        states = states.flatten()
        actions = actions.flatten()
        ax1.plot(states, colors['tab:blue'])
        ax1.plot([sp_list[i//sp_steps] for i in range(num_steps)]*num_episodes, colors['tab:orange'])
        ax2.plot(actions, colors['tab:green'])
        
        ax1.set(ylabel="SetPoint and state (kPa)", title="Trajectory")
        ax2.set(ylabel="Action u(t) /rpm", xlabel="Steps")

        plt.tight_layout()


class Environment:
    def __init__(self, alpha_u=0.1, reward_scale=0.1, *args, **kwargs):
        self.states = np.round(np.arange(-5, 5, 0.1), 2)
        self.state_space_low = self.states[0]
        self.state_space_high = self.states[-1]

        self.action_space = np.round(np.arange(-5, 5, 0.1), 2)
        self.action_space_low = 0
        self.action_space_high = 50

        self.num_actions = 1
        self.num_states = 1
        self.reward = 0.0
        self.state = 0.0
        self.action = 0.0
        self.alpha_u = alpha_u
        self.reward_scale = reward_scale
        self.SP = []

    def get_next_state(self, u):
        x = -2.1761 + -0.202*self.state + 0.014*u**2
        return x

    def reset(self):
        self.reward = 0.0
        self.state = 0.0
        self.action = 0.0
        return self.state

    def step(self, action, sp):
        
        # clip state space and calculate reward
        
        del_next_state = np.round(np.clip(self.get_next_state(action), self.state_space_low, self.state_space_high), 1)
        
        # get reward for un-normalized state and action as step is a function for un-normalized state and action
        self.reward = -self.reward_scale*(np.abs(sp-self.state) + self.alpha_u*np.abs(self.action-action))
        #print('state',self.state, 'action', action, 'next_state', next_state)
        
        self.state += del_next_state
        self.action = np.round(action, 1)

        # return normalized state
        return del_next_state, self.reward


# Define the main function
def Q_run():

    alpha = 0.133 #trial.suggest_loguniform('alpha', 0.0001, 1)
    gamma = 0.99 #trial.suggest_loguniform('gamma', 0.0001, 1)
    agent = QLearningAgent(alpha=alpha, gamma=gamma, epsilon=0.6, epsilon_decay=3/num_episodes,
                        alpha_decay=2/num_episodes, get_state=lambda: 0, action_space=env.action_space)
    agent.gen_Q_table(agent.action_space, env.states)


    
    for episode in range(num_episodes):
        state = env.reset()
        
        step = 0
        action = 0
        del_next_state = 0

        while state is not None and step < max_steps:
            SP = sp_list[step//(sp_steps+1)]

            del_action = agent.choose_action(del_next_state)
            action += del_action
            action = np.round(np.clip(action,0, 50), 1)
            

            del_next_state, reward = env.step(action, SP)
            state += del_next_state
            state = np.round(np.clip(state, -10, 40), 1)
            rewards[episode] += reward
           
            
            if episode%50 == 0:
              print(f'step: {step:5d}, state:{state:4.2f}, del_next_state:{del_next_state:4.2f}, action: {action:4.2f}')
              with summary_writer.as_default():
                tf.summary.scalar('trajectory', state, step=step+max_steps*episode//50)
              with summary_writer2.as_default():
                tf.summary.scalar('trajectory', SP, step=step+max_steps*episode//50)
                tf.summary.scalar('action', action, step=step+max_steps*episode//50)
              pass

            if del_next_state is None:
                print('del_next_state None')
                if state-(env.state_space_low+env.state_space_high)/2 < 0:
                    del_next_state = env.state_space_low
                else:
                    del_next_state = env.state_space_high
                 
            else:
                # continue the episode
                error[episode] += np.abs(SP-state)
            agent.update_Q_value(del_next_state, del_action, reward)

            step += 1
        # epsilon decay & alpha decay
        agent.epsilon = agent.epsilon*np.exp(-agent.epsilon_decay)
        agent.alpha = agent.alpha*np.exp(-agent.alpha_decay)
        
        if episode%10 == 0:
            print(f"Episode: {episode : 04d}/{num_episodes : 04d} | episodic reward: {rewards[episode]: 04.2f} |  error:{error[episode]: 04.2f}")
        
            with summary_writer.as_default():
                tf.summary.scalar('error', error[episode], step=episode)
                tf.summary.scalar('reward', rewards[episode], step=episode)
                
                a = tf.convert_to_tensor(agent.Q)
                img = np.reshape((a - tf.math.reduce_min(a))/(tf.math.reduce_max(a)-tf.math.reduce_min(a)), (-1, agent.Q.shape[0], agent.Q.shape[1], 1))
                tf.summary.image('Q table', img, step=episode)
                

        
    return agent
    


In [62]:

env = Environment()

# run the Q-learning algorithm for a fixed number of episodes
num_episodes = 10000
max_steps = 500
sp_steps = 50
alpha = 0
gamma = 0

log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))
if not os.path.exists('logs'):
    os.mkdir('logs')
    os.mkdir(log_dir)

summary_writer = tf.summary.create_file_writer(log_dir)
summary_writer2 = tf.summary.create_file_writer(log_dir+"_SP")

rewards = np.zeros(num_episodes)
error = np.zeros(num_episodes)
sp_list = [np.random.random()*35-5 for i in range(max_steps//sp_steps)]
#[20]*(max_steps//sp_steps)#
print("SP List: ",sp_list)

agent = Q_run()

Episode: 1/2000, episodic reward: -148275.88, set point: 10.4, error:243.46000000000006
Episode: 101/2000, episodic reward: -120903.19, set point: -0.5, error:385.03
Episode: 201/2000, episodic reward: -96306.69, set point: 2.3, error:527.47
Episode: 301/2000, episodic reward: -182641.52, set point: 14.6, error:-89.3299999999999
Episode: 401/2000, episodic reward: -104485.29, set point: 6.4, error:492.51
Episode: 501/2000, episodic reward: -259163.17, set point: 18.6, error:73.27000000000004
Episode: 601/2000, episodic reward: -98213.19, set point: 7.1, error:-260.71
Episode: 701/2000, episodic reward: -96722.23, set point: 1.5, error:1102.99
Episode: 801/2000, episodic reward: -184366.32, set point: 15.6, error:-893.4499999999998
Episode: 901/2000, episodic reward: -85827.26, set point: 1.7, error:824.8299999999999
Episode: 1001/2000, episodic reward: -86680.7, set point: 4.8, error:1389.08
Episode: 1101/2000, episodic reward: -102413.67, set point: 5.8, error:-1497.0
Episode: 1201/20